In [1]:
import os
import numpy as np
import tensorflow as tf

In [2]:
import re
file = open("data/enwik8_cleaned.txt", "r")
doclist = [line for line in file]
docstr = ''.join(doclist)
sentences = re.split(r'[.!?]', docstr)
sentences = [sentence.split() for sentence in sentences if len(sentence) > 1]

In [3]:
print(len(sentences))
#sentences_short = sentences[:150000]

841088


In [5]:
import tf_synt_randwalk
model_srw = tf_synt_randwalk.SyntRandWalkModel(embedding_size=150, context_size=5, min_occurrences=600,
                            learning_rate=0.05, batch_size=512)

In [6]:
model_srw.fit_to_corpus(sentences)

In [7]:
model_srw.train(num_epochs=5)

In [8]:
#if tensor was not already saved, load_from_file=False
#depend on context_size and corpus size
model_srw.fit_to_corpus_synt(sentences, "cooc_tensor.pkl", load_from_file=False)

Region 0 done.
Region 1000 done.
Region 2000 done.
Region 3000 done.
Region 4000 done.
Region 5000 done.
Region 6000 done.
Region 7000 done.
Region 8000 done.
Region 9000 done.
Region 10000 done.
Region 11000 done.
Region 12000 done.
Region 13000 done.
Region 14000 done.
Region 15000 done.
Region 16000 done.
Region 17000 done.
Region 18000 done.
Region 19000 done.
Region 20000 done.
Region 21000 done.
Region 22000 done.
Region 23000 done.
Region 24000 done.
Region 25000 done.
Region 26000 done.
Region 27000 done.
Region 28000 done.
Region 29000 done.
Region 30000 done.
Region 31000 done.
Region 32000 done.
Region 33000 done.
Region 34000 done.
Region 35000 done.
Region 36000 done.
Region 37000 done.
Region 38000 done.
Region 39000 done.
Region 40000 done.
Region 41000 done.
Region 42000 done.
Region 43000 done.
Region 44000 done.
Region 45000 done.
Region 46000 done.
Region 47000 done.
Region 48000 done.
Region 49000 done.
Region 50000 done.
Region 51000 done.
Region 52000 done.
Region

Region 415000 done.
Region 416000 done.
Region 417000 done.
Region 418000 done.
Region 419000 done.
Region 420000 done.
Region 421000 done.
Region 422000 done.
Region 423000 done.
Region 424000 done.
Region 425000 done.
Region 426000 done.
Region 427000 done.
Region 428000 done.
Region 429000 done.
Region 430000 done.
Region 431000 done.
Region 432000 done.
Region 433000 done.
Region 434000 done.
Region 435000 done.
Region 436000 done.
Region 437000 done.
Region 438000 done.
Region 439000 done.
Region 440000 done.
Region 441000 done.
Region 442000 done.
Region 443000 done.
Region 444000 done.
Region 445000 done.
Region 446000 done.
Region 447000 done.
Region 448000 done.
Region 449000 done.
Region 450000 done.
Region 451000 done.
Region 452000 done.
Region 453000 done.
Region 454000 done.
Region 455000 done.
Region 456000 done.
Region 457000 done.
Region 458000 done.
Region 459000 done.
Region 460000 done.
Region 461000 done.
Region 462000 done.
Region 463000 done.
Region 464000 done.


Region 824000 done.
Region 825000 done.
Region 826000 done.
Region 827000 done.
Region 828000 done.
Region 829000 done.
Region 830000 done.
Region 831000 done.
Region 832000 done.
Region 833000 done.
Region 834000 done.
Region 835000 done.
Region 836000 done.
Region 837000 done.
Region 838000 done.
Region 839000 done.
Region 840000 done.
Region 841000 done.
fit done
graph done


In [9]:
model_srw.train_synt(num_epochs=5)

Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
def vectorize_word(m, word):
    vec = None
    if isinstance(word, np.ndarray):
        vec = word
    if isinstance(word, str):
        vec = m.embedding_for(word)
    return vec

def nearest_words(m, word, top_n=10):
        vec = vectorize_word(m, word)
        vec = vec.reshape((1, -1))
        cosines = cosine_similarity(vec, m.embeddings)
        top10_ind = np.argsort(cosines)[0][::-1]
        neighbors = [(m.words[word_ind], cosines[0][top10_ind[i]]) for i, word_ind in enumerate(top10_ind)]
        return neighbors[1:top_n+1]

def nearest_words_to_pairs_addition(m, word_a, word_b, top_n=10):
    vec_a = vectorize_word(m, word_a)
    vec_b = vectorize_word(m, word_b)
    vec = vec_a + vec_b
    #print(vec.shape)
    vec = vec.reshape((1, -1))
    cosines = cosine_similarity(vec, m.embeddings)
    top10_ind = np.argsort(cosines)[0][::-1]
    neighbors = [(m.words[word_ind], cosines[0][top10_ind[i]]) for i, word_ind in enumerate(top10_ind)]
    return neighbors[1:top_n+1]

def nearest_words_to_pairs_tensor(m, word_a, word_b, top_n=10):
    vec_a = vectorize_word(m, word_a)
    vec_b = vectorize_word(m, word_b)
    vec = vec_a + vec_b + np.einsum('ijk,i,j->k', m.getT(), vec_a, vec_b)
    vec = vec.reshape((1, -1))
    cosines = cosine_similarity(vec, m.embeddings)
    top10_ind = np.argsort(cosines)[0][::-1]
    neighbors = [(m.words[word_ind], cosines[0][top10_ind[i]]) for i, word_ind in enumerate(top10_ind)]
    return neighbors[1:top_n+1]

def show_three(m_gv, m_srw, word_a, word_b):
    print("Word pair: {} {}".format(word_a, word_b))
    print("GloVe addition:\n", nearest_words_to_pairs_addition(m_gv, word_b, word_a), "\n")
    print("RandWalk addition:\n", nearest_words_to_pairs_addition(m_srw, word_b, word_a), "\n")
    print("SyntRandWalk tensor:\n", nearest_words_to_pairs_tensor(m_srw, word_b, word_a), "\n")

In [11]:
import tf_glove
model_gv = tf_glove.GloVeModel(embedding_size=150, context_size=5, min_occurrences=600,
                            learning_rate=0.05, batch_size=512)

In [12]:
model_gv.fit_to_corpus(sentences)

In [13]:
model_gv.train(num_epochs=10, log_dir="log/example_rw", summary_batch_interval=1000)

Writing TensorBoard summaries to log/example_rw


In [14]:
show_three(model_gv, model_srw, 'european', 'union')

Word pair: european union
GloVe addition:
 [('european', 0.80584663), ('state', 0.40747604), ('america', 0.36230046), ('zero', 0.35237995), ('lead', 0.35155404), ('record', 0.3479799), ('follow', 0.34129906), ('year', 0.32314456), ('kingdom', 0.31921986), ('force', 0.31865367)] 

RandWalk addition:
 [('european', 0.3785884), ('quarter', 0.25338805), ('iron', 0.240315), ('root', 0.23144813), ('mean', 0.22720034), ('reason', 0.22282484), ('channel', 0.22189891), ('treat', 0.21712597), ('pure', 0.21252073), ('environment', 0.21087052)] 

SyntRandWalk tensor:
 [('union', 0.249822), ('database', 0.24775966), ('quarter', 0.23817232), ('agricultural', 0.23109871), ('platform', 0.2185377), ('visit', 0.20479307), ('orthodox', 0.19977179), ('element', 0.19944623), ('specific', 0.19828194), ('conduct', 0.19692604)] 



In [16]:
show_three(model_gv, model_srw, 'soviet', 'union')

Word pair: soviet union
GloVe addition:
 [('union', 0.7999264), ('zero', 0.38695282), ('force', 0.38403356), ('work', 0.38369176), ('lead', 0.3432425), ('maintain', 0.34066224), ('establish', 0.32653376), ('seven', 0.31724748), ('trade', 0.31643683), ('form', 0.3137048)] 

RandWalk addition:
 [('soviet', 0.5488293), ('return', 0.2602409), ('lord', 0.23483501), ('inhabitant', 0.21918204), ('agency', 0.21860161), ('abandon', 0.21561676), ('classic', 0.21094984), ('variety', 0.20999669), ('reason', 0.20838718), ('airline', 0.20741442)] 

SyntRandWalk tensor:
 [('french', 0.24344033), ('replace', 0.22245368), ('thoma', 0.22046046), ('liquid', 0.2166215), ('context', 0.21517296), ('mechanic', 0.21392775), ('april', 0.21337318), ('crew', 0.21015991), ('import', 0.20998739), ('resident', 0.2048283)] 



In [17]:
show_three(model_gv, model_srw, 'civil', 'war')

Word pair: civil war
GloVe addition:
 [('war', 0.77538884), ('movement', 0.3348989), ('go', 0.3089383), ('world', 0.28871658), ('decline', 0.28724998), ('period', 0.28232846), ('follow', 0.27902576), ('fight', 0.2783193), ('ally', 0.27516258), ('religion', 0.27466914)] 

RandWalk addition:
 [('war', 0.3512283), ('kill', 0.26855534), ('publication', 0.24646825), ('legislative', 0.21189187), ('shoot', 0.20961982), ('polish', 0.20868519), ('track', 0.2065013), ('officer', 0.20602275), ('marine', 0.20147237), ('japan', 0.19471602)] 

SyntRandWalk tensor:
 [('find', 0.27075318), ('negative', 0.23664866), ('represent', 0.23639637), ('addition', 0.22897847), ('august', 0.22148417), ('read', 0.21823044), ('popularity', 0.21232131), ('isle', 0.2112494), ('interest', 0.2098148), ('multiple', 0.20582202)] 



In [19]:
show_three(model_gv, model_srw, 'united', 'state')

Word pair: united state
GloVe addition:
 [('united', 0.78091806), ('seven', 0.41201642), ('year', 0.4077241), ('president', 0.39152205), ('new', 0.38762358), ('nation', 0.38070655), ('union', 0.376728), ('national', 0.3763331), ('city', 0.36849248), ('canada', 0.362693)] 

RandWalk addition:
 [('org', 0.2661831), ('deal', 0.25081423), ('united', 0.24377227), ('manner', 0.22866495), ('spain', 0.22334598), ('violence', 0.21051168), ('price', 0.20896678), ('blood', 0.2071776), ('influence', 0.20571037), ('berlin', 0.20519798)] 

SyntRandWalk tensor:
 [('southern', 0.2480389), ('sexual', 0.24136783), ('prominent', 0.2315869), ('idea', 0.22690746), ('dispute', 0.22483176), ('successor', 0.22477953), ('wish', 0.22281007), ('classification', 0.21387102), ('dialect', 0.21386331), ('cover', 0.20958829)] 



In [24]:
show_three(model_gv, model_srw, 'complex', 'number')

Word pair: complex number
GloVe addition:
 [('number', 0.7574048), ('apply', 0.3991113), ('real', 0.38288426), ('consist', 0.35449475), ('theory', 0.34217727), ('consider', 0.33541054), ('function', 0.32705966), ('time', 0.32688373), ('word', 0.3117966), ('know', 0.31001538)] 

RandWalk addition:
 [('number', 0.6724603), ('channel', 0.27025017), ('exile', 0.22978692), ('prize', 0.2274992), ('treat', 0.226753), ('gay', 0.21612065), ('polish', 0.21370885), ('set', 0.20964159), ('license', 0.20625006), ('action', 0.205742)] 

SyntRandWalk tensor:
 [('professor', 0.22862509), ('turn', 0.22063227), ('display', 0.21936107), ('existence', 0.21782258), ('complex', 0.21762946), ('audience', 0.20450257), ('potential', 0.20198599), ('late', 0.20055416), ('grand', 0.19945845), ('fruit', 0.19924843)] 



In [22]:
show_three(model_gv, model_srw, 'algebra', 'linear')

Word pair: algebra linear
GloVe addition:
 [('linear', 0.7199656), ('fly', 0.2617682), ('pair', 0.25431967), ('instruction', 0.2528658), ('give', 0.25134242), ('hebrew', 0.24940987), ('good', 0.23839869), ('http', 0.23481546), ('andrew', 0.23472095), ('naval', 0.22946066)] 

RandWalk addition:
 [('linear', 0.6204195), ('soil', 0.26855025), ('understand', 0.2599407), ('satellite', 0.25758106), ('left', 0.23430094), ('depression', 0.23132311), ('berlin', 0.22591062), ('adopt', 0.22361018), ('drink', 0.22148737), ('mail', 0.2175842)] 

SyntRandWalk tensor:
 [('duke', 0.22770864), ('height', 0.2260373), ('compete', 0.22324912), ('angle', 0.22289035), ('fox', 0.21724656), ('mathematic', 0.21428159), ('type', 0.20962377), ('eat', 0.2043113), ('official', 0.20369323), ('picture', 0.2014814)] 



In [32]:
show_three(model_gv, model_srw, 'united', 'kingdom')

Word pair: united kingdom
GloVe addition:
 [('united', 0.80078286), ('canada', 0.3853266), ('year', 0.35412666), ('seven', 0.35188165), ('establish', 0.33784398), ('zero', 0.32822967), ('state', 0.3127894), ('prince', 0.31253093), ('election', 0.31045997), ('nation', 0.3062004)] 

RandWalk addition:
 [('kingdom', 0.6575733), ('establish', 0.2817736), ('farm', 0.28020647), ('town', 0.26814896), ('suffer', 0.259837), ('conservative', 0.25333872), ('unit', 0.2466909), ('beat', 0.2433917), ('continental', 0.23808342), ('troop', 0.23715155)] 

SyntRandWalk tensor:
 [('united', 0.25713536), ('billion', 0.23693413), ('temperature', 0.23092896), ('switch', 0.23003055), ('farm', 0.22946139), ('render', 0.22413509), ('concern', 0.2218587), ('aircraft', 0.20944993), ('diamond', 0.20533912), ('market', 0.20441657)] 



In [28]:
print(model_gv.time_fit_corp)
print(model_gv.time_train)
print(model_srw.time_rw_fit_corp)
print(model_srw.time_rw_train)
print(model_srw.time_srw_fit_corp)
print(model_srw.time_srw_train)

65.27105753102842
64.38091009123491
64.53209054227408
36.19991396924401
9506.720353154215
3805.0908896079654
